In [10]:
#only for testing
def return_question():
    import random
    next_ques=random.choice(["Yooooo!",'Heyyyy!','Weeeee!'])
    pic_link='https://bakingamoment.com/wp-content/uploads/2018/02/IMG_6207-chocolate-molten-lava-cakes-recipe-square.jpg'
    return next_ques,pic_link


from tkinter import *
from tkinter import scrolledtext
import urllib.request
from PIL import Image, ImageTk
from io import BytesIO

#functions to be implemented: connect each page to the randomly generated question
#user can choose from a multiple choice question
#user is told right or wrong
#count the correct answer number
#quit button
class quizGUI:
    
#     object_list = [{'Title': 'One-dollar Liberty Head Coin',
#   'Artist': 'James Barton Longacre',
#   'Period': '1853',
#   'Medium': 'Gold',
#   'Classification': 'Metal',
#   'Department': 'American Decorative Arts'},
#  {'Title': 'Ten-dollar Liberty Head Coin',
#   'Artist': 'Christian Gobrecht',
#   'Period': '1901',
#   'Medium': 'Gold',
#   'Classification': 'Metal',
#   'Department': 'American Decorative Arts'},
#  {'Title': 'Two-and-a-half-dollar Indian Head Coin',
#   'Artist': 'Bela Lyon Pratt',
#   'Period': '1912',
#   'Medium': 'Gold',
#   'Classification': 'Metal',
#   'Department': 'American Decorative Arts'},
#  {'Title': 'Two-and-a-half-dollar Liberty Head Coin',
#   'Artist': 'Christian Gobrecht',
#   'Period': '1907',
#   'Medium': 'Gold',
#   'Classification': 'Metal',
#   'Department': 'American Decorative Arts'},
#  {'Title': 'Twenty-dollar Liberty Head Coin',
#   'Artist': 'James Barton Longacre',
#   'Period': '1876',
#   'Medium': 'Gold',
#   'Classification': 'Metal',
#   'Department': 'American Decorative Arts'},
#  {'Title': 'Five-dollar Indian Head Coin',
#   'Artist': 'Bela Lyon Pratt',
#   'Period': '1910',
#   'Medium': 'Gold',
#   'Classification': 'Metal',
#   'Department': 'American Decorative Arts'}]
#     avai_obj = len(object_list)-1
#     rand_obj = random.randint(0, avai_obj)  # Get a random object number
#     correct_answer = object_list[rand_obj]
#     wrong = [object_department_list[i] for i in random.sample(range(1, len(department_of_obj)), 3) and i != obj_number]
#     w1 = wrong[0]
#     w2 = wrong[1]
#     w3 = wrong[2]
#     Q = random.randint(0,4)
    
#     def questions(category, obj):
#         if category is 'artist':
#             Q = 0
#             question = "Who is the artist for this piece of work?"
#             correct = obj.get_artist()
#             wrong1 = w1.get_artist()
#             wrong2 = w2.get_artist()
#             wrong3 = w3.get_artist()
#         elif category is 'time':
#             Q = 1
#             question = "During which time period was this artwork created?"
#             correct = obj.get_time()
#             wrong1 = w1.get_time()
#             wrong2 = w2.get_time()
#             wrong3 = w3.get_time()
#         elif category is 'material':
#             Q = 2
#             question = "What kind of main material did this object use?"
#             correct = obj.get_material()
#             wrong1 = w1.get_material()
#             wrong2 = w2.get_material()
#             wrong3 = w3.get_material()
#         elif category is 'type':
#             Q = 3
#             question = "What type of artwork is this?"
#             correct = obj.get_type()
#             wrong1 = w1.get_type()
#             wrong2 = w2.get_type()
#             wrong3 = w3.get_type()
#         else: #department
#             question = "Which department does this exhibit belong to?"
#             correct = obj.get_department()
#             wrong1 = w1.get_department()
#             wrong2 = w2.get_department()
#             wrong3 = w3.get_department()
    
#     answer_list = [wrong1,wrong2,wrong3,correct]
#     index_list = ['A','B','C','D']
#     random.shuffle(answer_list)
#     answer = dict(zip(index_list,answer_list))
#     answer_key = [key for key, value in answer.items() if value == correct]
    
    def __init__(self):
        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("800x600")
        root.title("MET-Q")
        self.root = root
        
        # Build GUI
        canvas_width = 800
        canvas_height = 600
        canvas = Canvas(root, height=canvas_height, width=canvas_width)
        
        #correct/incorrect indicator
        var=StringVar()
        self.cor_label=Label(root,textvariable=var)
        self.var=var
        self.cor_label.pack()
        #title label
        self.title_label = Label(root, text="The MET Quiz", font = "Times 20 bold").pack(side="top")
    
        #record the number of correct answers so far
        self.total = IntVar() #total correct answers
        self.total_var=StringVar()
        self.total_var.set('Number of correct answers: 0')
        self.score_label = Label(root,textvariable=self.total_var)
        self.score_label.pack()
        
        #record the number of questions completed so far
        self.num_var=StringVar()
        self.num=IntVar()
        self.num_var.set('Questions completed: 0')
        self.num_of_ques=Label(root,textvariable=self.num_var)
        self.num_of_ques.pack()
    
        #question stem
        obj_title = StringVar()  # object string
        question_var=StringVar() # question string
        self.Q = Label(root, textvariable=question_var).pack() #question label
        question_var.set(return_question()[0])
        self.question_var=question_var
        self.obj_label = Label(root, textvariable=obj_title).pack() #add object title
        
#         Question picture
        URL=return_question()[1]
        u = urllib.request.urlopen(URL)
        raw_data = u.read()
        u.close()
        image_file = Image.open(BytesIO(raw_data))
        photo_image = ImageTk.PhotoImage(image_file)
        self.pic = Label(image=photo_image)
        self.pic.pack()
        
        #initialize choice buttons
        self.A = Button(root, text="A",command=lambda: self.correct())
        self.A.pack(padx=5,pady=5)
        self.B = Button(root, text="B",command=lambda: self.incorrect())
        self.B.pack(padx=5,pady=5)
        self.C = Button(root, text="C",command=lambda: self.incorrect())
        self.C.pack(padx=5,pady=5)
        self.D = Button(root, text="D",command=lambda: self.incorrect())
        self.D.pack(padx=5,pady=5)
        self.next=Button(root, text="Next question",command=lambda:self.next_()).pack()

        #quit button
        self.close_button=Button(root, text="Quit", command=lambda: root.destroy())
        self.close_button.pack(padx=20,side='right')
        
        frame=Frame(root)
        frame.pack()
        canvas.pack()
        
        root.mainloop() 
        

    #add 1 to score if the correct answer is chosen
    def correct(self):
        self.var.set('Correct')
        self.total.set(self.total.get() + 1) 
        self.total_var.set('Number of correct answers: '+str(self.total.get()))
    
    #display text when incorrect answer is chosen
    def incorrect(self):
        self.var.set('Incorrect')
        
    #generate another question and set up random question and choices
    #refer to create questions file
#     def display_next_question(self, self.total, self.question, self.obj_title, self.A, self.B, self.C, self.D)
#            wrong_left = 2
#          for i in range(4):
#             if i == 0:
#                 if correct_index == i:
#                     self.A.configure(text='A: '+correct_answer, command=lambda: correct())
#                 else:
#                     self.A.configure(text='A: '+wrong[wrong_left], command=lambda: incorrect())
#                     wrong_left -= 1
#             elif i == 1:
#                 if correct_index == i:
#                     self.B.configure(text='B: '+correct_answer, command=lambda: correct())
#                 else:
#                     self.B.configure(text='B: '+wrong[wrong_left], command=lambda: incorrect())
#                     wrong_left -= 1
#             elif i == 2:
#                 if correct_index == i:
#                     self.C.configure(text='C: '+correct_answer, command=lambda: correct())
#                 else:
#                     self.C.configure(text='C: '+wrong[wrong_left], command=lambda: incorrect())
#                     wrong_left -= 1
#             elif i == 3:
#                 if correct_index == i:
#                     self.D.configure(text='D: '+correct_answer, command=lambda: correct())
#                 else:
#                     self.D.configure(text='D: '+wrong[wrong_left], command=lambda: incorrect())
#                     wrong_left -= 1
    
        
    def next_(self):
        self.question_var.set(return_question()[0])
        self.num.set(self.num.get()+1)
        self.num_var.set('Questions completed: '+str(self.num.get()))

       

In [11]:
myGUI = quizGUI()

ValueError: unknown resampling filter